In [1]:
import pandas as pd
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\henrique\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def palvavra_separadas(dados, len_palavras = 2):
  '''
  ******************************************************************************
  palvavra_separadas: Separa as palavras com mais de
  len_palavras caracteres, retirando pontuacoes.
  ------------------------------------------------------------------------------
  Input:
  ------------------------------------------------------------------------------
  dados: Datafame
  len_palavras : tamanho minimo das palavras que seram considerado (>2)
  ------------------------------------------------------------------------------
  Outout:
  ------------------------------------------------------------------------------
  tokens: listas com as palavras separadas
  ------------------------------------------------------------------------------
  ******************************************************************************
  '''
  tokens = []
  for frase in dados["tratados"]:
    for token in nltk.tokenize.word_tokenize(frase):
      if token.isalpha() and len(token) > len_palavras:
        tokens.append(token.lower())

  return tokens

def gera_palavras_unicas(palavras):
  '''
  ******************************************************************************
  gera_palavras_unicas: Gera conjunto com palavras unicas.
  ------------------------------------------------------------------------------
  Input:
  ------------------------------------------------------------------------------
  palavra: lista com as palavras  
  ------------------------------------------------------------------------------
  Outout:
  ------------------------------------------------------------------------------
  tokens: listas com as palavras separadas
  ------------------------------------------------------------------------------
  ******************************************************************************
  '''
  unicas = set()

  for palavra in palavras:
    unicas.add(palavra)

  return unicas

In [3]:
def substituir(dados):
  '''
  ******************************************************************************
  Tratando a base de dados.
  ------------------------------------------------------------------------------
  Input:
  ------------------------------------------------------------------------------
  dados: Datafame original
  ------------------------------------------------------------------------------
  Outout:
  ------------------------------------------------------------------------------
  dados: Datafame com coluna tratado
  ------------------------------------------------------------------------------
  ******************************************************************************
  '''
  dados["tratados"] = dados["front"]

  retirar = re.compile("\[anki:play:q:0\]")
  linhas = []
  for linha in dados["tratados"]:    
    linhas.append(re.sub(retirar, '', linha))
  dados["tratados"] = linhas

  retirar = re.compile("\[Grammar\]")
  linhas = []
  for linha in dados["tratados"]:    
    linhas.append(re.sub(retirar, '', linha))
  dados["tratados"] = linhas

  retirar = re.compile("\[Inglês Britânico\]")
  linhas = []
  for linha in dados["tratados"]:    
    linhas.append(re.sub(retirar, '', linha))
  dados["tratados"] = linhas

In [42]:
def make_date(file_name):
  '''
  ******************************************************************************
  make_date: Le o nome do arquivo e retorna a data.
  ------------------------------------------------------------------------------
  Input:
  ------------------------------------------------------------------------------
  file_name: nome do arquivo
  ------------------------------------------------------------------------------
  Outout:
  ------------------------------------------------------------------------------
  Data no formato 00/00/0000
  ------------------------------------------------------------------------------
  ******************************************************************************
  '''
  return file_name[file_name.index("anki")+5:file_name.index(".")].replace("_","/")

In [43]:
files = "dados/anki_11_02_2021.txt", "dados/anki_20_02_2021.txt", "dados/anki_08_03_2021.txt"

In [44]:
for file_name in files:
  dados = pd.read_csv(file_name, sep="\t", header=None, names = ["front", "back"])
  substituir(dados)
  palavras = palvavra_separadas(dados)
  total_palavras = len(palavras)
  palavras_unicas = gera_palavras_unicas(palavras)
  total_de_palavras_unicas = len(palavras_unicas)
  print(f"data                     : {make_date(file_name)}")
  print(f"Total de palavras        : {total_palavras}")
  print(f"Total de palavras unicas : {total_de_palavras_unicas}")
  print()

data                     : 11/02/2021
Total de palavras        : 22722
Total de palavras unicas : 4862

data                     : 20/02/2021
Total de palavras        : 23838
Total de palavras unicas : 4980

data                     : 08/03/2021
Total de palavras        : 27007
Total de palavras unicas : 5396

